In [669]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames[:3]:
        print(os.path.join(dirname, filename))
    if len(filenames) > 3:
        print("...")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/captcha-hacker/sample_submission.csv
/kaggle/input/captcha-hacker/test/task1/x4LPcV5n6IvLj4vz.png
/kaggle/input/captcha-hacker/test/task1/W88fVMlAs5IpsXn4.png
/kaggle/input/captcha-hacker/test/task1/ZWDL6pUMfPu5c9jh.png
...
/kaggle/input/captcha-hacker/test/task2/jMalnsI5a5IWxYAi.png
/kaggle/input/captcha-hacker/test/task2/ihE9HHgyOINGEMcO.png
/kaggle/input/captcha-hacker/test/task2/ZATEVW3P5s0akZjd.png
...
/kaggle/input/captcha-hacker/test/task3/cXBlxYfvQWbiK7dn.png
/kaggle/input/captcha-hacker/test/task3/5gEp1jR9jNNfuqlk.png
/kaggle/input/captcha-hacker/test/task3/hEQ0WQtB9B7j8C2f.png
...
/kaggle/input/captcha-hacker/train/annotations.csv
/kaggle/input/captcha-hacker/train/task1/H85RQ6dbWUvLSIDV.png
/kaggle/input/captcha-hacker/train/task1/n2GC8uY1N4QfvVxe.png
/kaggle/input/captcha-hacker/train/task1/XOqfRx2R6SnoEjFr.png
...
/kaggle/input/captcha-hacker/train/task2/Mr4B2zxXk92hyzn9.png
/kaggle/input/captcha-hacker/train/task2/SIuRCnlK8VS91FhX.png
/kaggle/input/captcha-h

In [670]:
import csv
import cv2
import numpy as np
import random
import tensorflow as tf
import os
import torchvision
import torchvision.transforms as transforms
from torchvision import io
from tqdm import tqdm
import torchvision.models as models
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [671]:
TRAIN_PATH = "/kaggle/input/captcha-hacker/train"
TEST_PATH = "/kaggle/input/captcha-hacker/test"
device = "cuda"
# try device = "cuda" 
# and change your settings/accelerator to GPU if you want it to run faster

In [673]:
transform_1= transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [674]:
transform_2= transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    #transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [675]:
transform_3 = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((384,288)),
    #384*288
    #transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [676]:
class Task1Dataset(Dataset):
    def __init__(self, data, root, return_filename=False):
        self.data = [sample for sample in data if sample[0].startswith("task1")]
        self.return_filename = return_filename
        self.root = root
    
    def __getitem__(self, index):
        filename, label = self.data[index]
        img = cv2.imread(f"{self.root}/{filename}")
#         img = cv2.resize(img, (32, 32))
#         img = np.mean(img, axis=2)
#         print(img.shape)
        img = transform_1(img)
        if self.return_filename:
            return torch.FloatTensor(img / 256), filename
        else:
            return torch.FloatTensor(img / 256), int(label)

    def __len__(self):
        return len(self.data)

In [677]:
class Task2Dataset(Dataset):
    def __init__(self, data, root, return_filename=False):
        self.data = [sample for sample in data if sample[0].startswith("task2")]
        self.return_filename = return_filename
        self.root = root
    
    def __getitem__(self, index):
        filename, label = self.data[index]
        img = cv2.imread(f"{self.root}/{filename}")
        #img = io.read_image(f"{self.root}/{filename}")
        #img = cv2.resize(img, (224, 224))
        img = transform_2(img)
        if self.return_filename:
            return torch.FloatTensor((img ) / 255), filename
        else:
            return torch.FloatTensor((img ) / 255), label

    def __len__(self):
        return len(self.data)

In [678]:
class Task3Dataset(Dataset):
    def __init__(self, data, root, return_filename=False):
        self.data = [sample for sample in data if sample[0].startswith("task3")]
        self.return_filename = return_filename
        self.root = root
    
    def __getitem__(self, index):
        filename, label = self.data[index]
        img = cv2.imread(f"{self.root}/{filename}")
        #img = io.read_image(f"{self.root}/{filename}")
        #img = cv2.resize(img, (224, 224))
        img = transform_3(img)
        if self.return_filename:
            return torch.FloatTensor((img ) / 255), filename
        else:
            return torch.FloatTensor((img ) / 255), label

    def __len__(self):
        return len(self.data)

In [679]:
class ResNet18_1(nn.Module):
    def __init__(self):
        super(ResNet18_1, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(512, 10)

    def forward(self, x):
        logits = self.model(x)
        return logits

In [680]:
class ResNet18_2(nn.Module):
    def __init__(self):
        super(ResNet18_2, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(512, 72)

    def forward(self, x):
        logits = self.model(x)
        return logits

In [681]:
class ResNet18_3(nn.Module):
    def __init__(self):
        super(ResNet18_3, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(512, 144)

    def forward(self, x):
        logits = self.model(x)
        return logits

In [682]:
train_data = []
val_data = []

file = open(f'{TRAIN_PATH}/annotations.csv', newline='')
with file as csvfile:
    for row in csv.reader(csvfile, delimiter=','):
        if random.random() < 0.7:
            train_data.append(row)
        else:
            val_data.append(row)
            
train_ds = Task1Dataset(train_data, root=TRAIN_PATH)
train_dl = DataLoader(train_ds, batch_size=64, num_workers=4, drop_last=True, shuffle=True)

val_ds = Task1Dataset(val_data, root=TRAIN_PATH)
val_dl = DataLoader(val_ds, batch_size=64, num_workers=4, drop_last=False, shuffle=False)

/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [683]:
model = ResNet18_1().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()
PATH_1 = "model_1.pt"

for epoch in range(10):
    print(f"Epoch [{epoch}]")
    model.train()
    for image, label in train_dl:
        image = image.to(device)
        label = label.to(device)
        #print(image.shape)
        
        pred = model(image)
        loss = loss_fn(pred, label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    sample_count = 0
    correct_count = 0
    model.eval()
    for image, label in val_dl:
        image = image.to(device)
        label = label.to(device)
        #image = image.permute(0,3,1,2)
        pred = model(image)
        loss = loss_fn(pred, label)
        
        pred = torch.argmax(pred, dim=1)
        
        sample_count += len(image)
        correct_count += (label == pred).sum()
        
    print("accuracy (validation):", correct_count / sample_count)
torch.save(model.state_dict(), PATH_1)

Epoch [0]
accuracy (validation): tensor(0.0961, device='cuda:0')
Epoch [1]
accuracy (validation): tensor(0.0720, device='cuda:0')
Epoch [2]
accuracy (validation): tensor(0.2624, device='cuda:0')
Epoch [3]
accuracy (validation): tensor(0.9897, device='cuda:0')
Epoch [4]
accuracy (validation): tensor(0.9966, device='cuda:0')
Epoch [5]
accuracy (validation): tensor(0.9949, device='cuda:0')
Epoch [6]
accuracy (validation): tensor(0.9983, device='cuda:0')
Epoch [7]
accuracy (validation): tensor(1., device='cuda:0')
Epoch [8]
accuracy (validation): tensor(1., device='cuda:0')
Epoch [9]
accuracy (validation): tensor(1., device='cuda:0')


In [685]:
train_ds = Task2Dataset(train_data, root=TRAIN_PATH)
train_dl = DataLoader(train_ds, batch_size=32, num_workers=2, drop_last=True, shuffle=True)

val_ds = Task2Dataset(val_data, root=TRAIN_PATH)
val_dl = DataLoader(val_ds, batch_size=32, num_workers=2, drop_last=False, shuffle=False)

model = ResNet18_2().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MultiLabelSoftMarginLoss()

PATH_2 = "model_2.pt"
for epoch in range(15):
    print(f"Epoch [{epoch}]")
    model.train()
    for image, label in train_dl:
        tt = list()
        #print (label)
        for i in label :
            temp = np.zeros(72)
            count=0
            for j in i:
                if count==0:
                    if(ord(j)>90):
                        temp[ord(j)-87]=1
                    else:
                        temp[int(j)]=1
                else:
                    
                    if(ord(j)>90):
                        temp[ord(j)-87+36]=1
                    else:
                        temp[int(j)+36]=1
                count+=1
            tt.append(temp)
        tt = np.array(tt)
        tt=torch.from_numpy(tt)
        tt=tt.to(device)
        image = image.to(device)
        #print(image.shape)
        #image = image.permute(0,3,1,2)
        pred = model(image)
        loss = loss_fn(pred, tt)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    sample_count = 0
    correct_count = 0
    model.eval()
    for image, label in val_dl:

        tt = list()
        #print (label)
        for i in label :
            temp = np.zeros(72)
            count=0
            for j in i:
                if count==0:
                    if(ord(j)>90):
                        temp[ord(j)-87]=1
                    else:
                        temp[int(j)]=1
                else:
                    if(ord(j)>90):
                        temp[ord(j)-87+36]=1
                    else:
                        temp[int(j)+36]=1
                count+=1
            tt.append(temp)
        tt = np.array(tt)
        ttt=torch.from_numpy(tt)
        ttt=ttt.to(device)
        image = image.to(device)
        #image = image.permute(0,3,1,2)
        pred = model(image)
        #print(pred)
        loss = loss_fn(pred, ttt)
        
        pred = pred.data.cpu().numpy()
        tt1 = list()
        for j in range(pred.shape[0]):
            temp = np.zeros(72)
            max=-10000
            index=0
            for i in range(36):
                if pred[j][i] > max:
                    index=i
                    max=pred[j][i]
            temp[index]=1
            
            max=-10000
            index=0
            for i in range(36):
                if pred[j][i+36] > max:
                    index=i+36
                    max=pred[j][i+36]
            temp[index]=1
            tt1.append(temp)
        tt1 = np.array(tt1)
        
        #print (tt1[0])
        #print (pred[0][0])
        #print (pred)
        sample_count += len(image)
        for i in range(tt.shape[0]):
            #print (tt[i]-tt1[i])
            #print ("\n")
            if ((tt[i]==tt1[i]).all()):
                correct_count += 1
        
    
    #print (sample_count)
    print("accuracy (validation):", correct_count / sample_count)
torch.save(model.state_dict(), PATH_2)

Epoch [0]
accuracy (validation): 0.0
Epoch [1]
accuracy (validation): 0.9577278731836195
Epoch [2]
accuracy (validation): 0.9947159841479525
Epoch [3]
accuracy (validation): 0.9933949801849405
Epoch [4]
accuracy (validation): 0.9920739762219286
Epoch [5]
accuracy (validation): 0.9749009247027741
Epoch [6]
accuracy (validation): 0.9458388375165125
Epoch [7]
accuracy (validation): 0.9788639365918098
Epoch [8]
accuracy (validation): 0.9630118890356671
Epoch [9]
accuracy (validation): 0.9933949801849405
Epoch [10]
accuracy (validation): 0.9973579920739762
Epoch [11]
accuracy (validation): 1.0
Epoch [12]
accuracy (validation): 1.0
Epoch [13]
accuracy (validation): 0.9920739762219286
Epoch [14]
accuracy (validation): 1.0


In [687]:
train_ds = Task3Dataset(train_data, root=TRAIN_PATH)
train_dl = DataLoader(train_ds, batch_size=32, num_workers=2, drop_last=True, shuffle=True)

val_ds = Task3Dataset(val_data, root=TRAIN_PATH)
val_dl = DataLoader(val_ds, batch_size=32, num_workers=2, drop_last=False, shuffle=False)

model = ResNet18_3().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.MultiLabelSoftMarginLoss()

PATH_3 = "model_3.pt"
for epoch in range(30):
    print(f"Epoch [{epoch}]")
    model.train()
    for image, label in train_dl:
        tt = list()
        #print (label)
        for i in label :
            temp = np.zeros(144)
            count=0
            for j in i:
                if count==0:
                    if(ord(j)>90):
                        temp[ord(j)-87]=1
                    else:
                        temp[int(j)]=1
                elif count==1:
                    if(ord(j)>90):
                        temp[ord(j)-87+36]=1
                    else:
                        temp[int(j)+36]=1
                elif count==2:
                    if(ord(j)>90):
                        temp[ord(j)-87+72]=1
                    else:
                        temp[int(j)+72]=1
                elif count==3:
                    if(ord(j)>90):
                        temp[ord(j)-87+108]=1
                    else:
                        temp[int(j)+108]=1
                count+=1
            tt.append(temp)
        tt = np.array(tt)
        tt=torch.from_numpy(tt)
        tt=tt.to(device)
        image = image.to(device)
        #image = image.permute(0,3,1,2)
        #print(image.shape)
        pred = model(image)
        loss = loss_fn(pred, tt)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    sample_count = 0
    correct_count = 0
    model.eval()
    for image, label in val_dl:
        tt = list()
        for i in label :
            temp = np.zeros(144)
            count=0
            for j in i:
                if count==0:
                    if(ord(j)>90):
                        temp[ord(j)-87]=1
                    else:
                        temp[int(j)]=1
                elif count==1:
                    if(ord(j)>90):
                        temp[ord(j)-87+36]=1
                    else:
                        temp[int(j)+36]=1
                elif count==2:
                    if(ord(j)>90):
                        temp[ord(j)-87+72]=1
                    else:
                        temp[int(j)+72]=1
                elif count==3:
                    if(ord(j)>90):
                        temp[ord(j)-87+108]=1
                    else:
                        temp[int(j)+108]=1
                count+=1
            tt.append(temp)
        tt = np.array(tt)
        ttt=torch.from_numpy(tt)
        ttt=ttt.to(device)
        image = image.to(device)
        #image = image.permute(0,3,1,2)
        #print(image.shape)
        pred = model(image)
        #print(pred)
        loss = loss_fn(pred, ttt)
        pred = pred.data.cpu().numpy()
        tt1 = list()
        for j in range(pred.shape[0]):
            temp = np.zeros(144)
            max=-10000
            index=0
            for i in range(36):
                if pred[j][i] > max:
                    index=i
                    max=pred[j][i]
            temp[index]=1
            max=-10000
            index=0
            for i in range(36):
                if pred[j][i+36] > max:
                    index=i+36
                    max=pred[j][i+36]
            temp[index]=1
            max=-10000
            index=0
            for i in range(36):
                if pred[j][i+72] > max:
                    index=i+72
                    max=pred[j][i+72]
            temp[index]=1
            max=-10000
            index=0
            for i in range(36):
                if pred[j][i+108] > max:
                    index=i+108
                    max=pred[j][i+108]
            temp[index]=1
            tt1.append(temp)
            #print(temp)
        tt1 = np.array(tt1)
        #print(tt1)
        #print (tt1[0])
        #print (pred[0][0])
        #print (pred)
        sample_count += len(image)
        for i in range(tt.shape[0]):
            #print (tt[i]-tt1[i])
            #print ("\n")
            if ((tt[i]==tt1[i]).all()):
                correct_count += 1
        
    print (sample_count)
    print("accuracy (validation):", correct_count / sample_count)
torch.save(model.state_dict(), PATH_3)

Epoch [0]
918
accuracy (validation): 0.0
Epoch [1]
918
accuracy (validation): 0.0
Epoch [2]
918
accuracy (validation): 0.0
Epoch [3]
918
accuracy (validation): 0.08823529411764706
Epoch [4]
918
accuracy (validation): 0.3137254901960784
Epoch [5]
918
accuracy (validation): 0.6677559912854031
Epoch [6]
918
accuracy (validation): 0.8213507625272332
Epoch [7]
918
accuracy (validation): 0.9041394335511983
Epoch [8]
918
accuracy (validation): 0.9193899782135077
Epoch [9]
918
accuracy (validation): 0.9215686274509803
Epoch [10]
918
accuracy (validation): 0.9411764705882353
Epoch [11]
918
accuracy (validation): 0.9400871459694989
Epoch [12]
918
accuracy (validation): 0.9575163398692811
Epoch [13]
918
accuracy (validation): 0.9564270152505446
Epoch [14]
918
accuracy (validation): 0.9629629629629629
Epoch [15]
918
accuracy (validation): 0.9684095860566448
Epoch [16]
918
accuracy (validation): 0.9694989106753813
Epoch [17]
918
accuracy (validation): 0.9651416122004357
Epoch [18]
918
accuracy (val